In [5]:
# %load main.py
import argparse
import time
import os
import os.path

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision import transforms

import PIL
from PIL import Image
import matplotlib.pyplot as plt
from collections import OrderedDict
from utility.utility import postp, GramMatrix, GramMSELoss, load_images, save_images, make_folders
from utility.vgg_network import VGG
from utility.loss_fns import get_patches, divide_patches, weight_maker, get_style_patch_weights, mrf_loss_fn

In [2]:
# style weights
sw1=1
sw2=1
sw3=1
sw4=1
sw5=1
# Content weights
cw1=0
cw2=0
cw3=0
cw4=1e5
cw5=0
#############################################################################
# Get image paths and names
# Style 1
style_dir1  = os.path.dirname('../input/font_contents/serif/A/Milonga-Regular.png')
style_name1 = os.path.basename('../input/font_contents/serif/A/Milonga-Regular.png')
# Style 2
style_dir2  = os.path.dirname('../input/font_contents/serif_rmv/A/Milonga-Regular.png')
style_name2 = os.path.basename('../input/font_contents/serif_rmv/A/Milonga-Regular.png')
# Content
content_dir  = os.path.dirname('../input/font_contents/AlegreyaSans-Light/A.png')
content_name = os.path.basename('../input/font_contents/AlegreyaSans-Light/A.png')

# Cuda device
if torch.cuda.is_available:
    device = 'cuda:0'
else:
    device = 'cpu'
print("Using device: ", device)

# Parameters
alpha = 1
beta = 1
image_size = 256
content_invert = 1
style_invert = 1
result_invert = content_invert

# Get output path
output_path = '../output_style_difference/'
try:
    os.mkdir(output_path)
except:
    pass
output_path = output_path + content_name[:-4] + '_' + style_name1[:-4] + '_' + style_name2[:-4] + '/'

Using device:  cuda:0


In [3]:
# Get network
vgg = VGG()
vgg.load_state_dict(torch.load('../Models/vgg_conv.pth'))
for param in vgg.parameters():
    param.requires_grad = False
if torch.cuda.is_available():
    vgg.cuda()

# Load images
content_image = load_images(os.path.join(content_dir, content_name), image_size, device, content_invert)
style_image1  = load_images(os.path.join(style_dir1,style_name1), image_size, device, style_invert)
style_image2  = load_images(os.path.join(style_dir2,style_name2), image_size, device, style_invert)

# Random input
# opt_img = Variable(torch.randn(content_image.size()).type_as(content_image.data).to(device), requires_grad=True).to(device)
# Content input
opt_img = Variable(content_image.data.clone(), requires_grad=True)

In [11]:
mrf_layers = ['r31', 'r41']

In [58]:
mrf_fms1 = [A.detach() for A in vgg(style_image1, mrf_layers)]
style_patches_lists1, weight_list1 = get_style_patch_weights(mrf_fms1, device, k=5)

In [59]:
mrf_fms2 = [A.detach() for A in vgg(style_image2, mrf_layers)]
style_patches_lists2, weight_list2 = get_style_patch_weights(mrf_fms2, device, k=5)

Finding and storing different patches only

In [63]:
style1_different_patches = []
style2_different_patches = []
i = 0
for p1, p2 in zip(style_patches_lists1[0], style_patches_lists2[0]):
    if not torch.eq(p1,p2).all():
        style1_different_patches.append(p1)
        style2_different_patches.append(p2)

In [64]:
len(style_patches_lists1[0]), len(style1_different_patches)

(3481, 663)

Finding difference between all corresponding patches

In [75]:
a = [p1-p2 for p1,p2 in zip(style_patches_lists1[0],style_patches_lists2[0])]
b = [p1-p2 for p1,p2 in zip(style_patches_lists1[1],style_patches_lists2[1])]

In [80]:
nn.MSELoss()(a[0],a[0])

tensor(0., device='cuda:0')